In [ ]:
!pip install -q kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d gti-upm/leapgestrecog
!unzip leapgestrecog.zip


Streaming output truncated to the last 5000 lines.
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0001.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0002.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0003.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0004.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0005.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0006.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0007.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0008.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0009.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0010.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0011.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0012.png  
  inflating: leapgestrecog/leapGestRe

In [ ]:
!pip install tensorflow opencv-python matplotlib scikit-learn


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras .applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt


In [ ]:
data_dir =  "/content/leapGestRecog"

= 64IMG_SIZE


In [ ]:
os.listdir(data_dir)


['04', '07', '03', '08', '09', '05', '01', '00', '06', '02']

In [ ]:
for person in os.listdir(DATA_DIR):              # 00, 01, 02...
    person_path = os.path.join(DATA_DIR, person)

    for gesture in os.listdir(person_path):      # 01_palm, 10_down...
        gesture_path = os.path.join(person_path, gesture)

        for img_name in os.listdir(gesture_path):
            img_path = os.path.join(gesture_path, img_name)
            img = cv2.imread(img_path)

            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            data.append(img)
            labels.append(gesture)

In [ ]:
data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
labels = np.array(labels)

gesture_names = sorted(list(set(labels)))
label_map = {gesture: idx for idx, gesture in enumerate(gesture_names)}

labels = np.array([label_map[label] for label in labels])

print("Gestures:", gesture_names)
print("Dataset shape:", data.shape)


Gestures: [np.str_('01_palm'), np.str_('02_l'), np.str_('03_fist'), np.str_('04_fist_moved'), np.str_('05_thumb'), np.str_('06_index'), np.str_('07_ok'), np.str_('08_palm_moved'), np.str_('09_c'), np.str_('10_down')]
Dataset shape: (20000, 64, 64, 1)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(gesture_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,275,146 (4.86 MB)

 Trainable params: 1,275,146 (4.86 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=64,
    validation_data=(X_test, y_test)
)


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 124s 483ms/step - accuracy: 0.7295 - loss: 0.7968 - val_accuracy: 0.9992 - val_loss: 0.0045
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 128s 511ms/step - accuracy: 0.9976 - loss: 0.0090 - val_accuracy: 0.9980 - val_loss: 0.0079
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 122s 489ms/step - accuracy: 0.9997 - loss: 0.0021 - val_accuracy: 0.9800 - val_loss: 0.0703
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 121s 484ms/step - accuracy: 0.9963 - loss: 0.0179 - val_accuracy: 0.9998 - val_loss: 8.2552e-04
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 140s 477ms/step - accuracy: 0.9989 - loss: 0.0023 - val_accuracy: 1.0000 - val_loss: 7.3156e-05
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 139s 467ms/step - accuracy: 1.0000 - loss: 1.8323e-04 - val_accuracy: 0.9983 - val_loss: 0.0053
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 122s 487ms/step - accuracy: 0.9968 - loss: 0.0109 - val_accuracy: 0.9995 - val_loss: 0.0015
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 119s 475ms/step - accura

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("✅ Test Accuracy:", test_acc)


125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.9996 - loss: 2.6959e-04
✅ Test Accuracy: 0.999750018119812


In [ ]:
model.save("hand_gesture_model.h5")
print("✅ Model saved as hand_gesture_model.h5")


✅ Model saved as hand_gesture_model.h5


In [ ]:
from google.colab import files
files.download("hand_gesture_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>